In [ ]:
#Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

In [ ]:
#Load the data
data = pd.read_parquet("yellow_tripdata_2024-08.parquet")

#Display the first 5 rows
data.head()

In [ ]:
#Remove rows with missing or zero passenger counts, fare amounts and trip distances
cleaned_data = data[
    (data['passenger_count'] > 0) &
    (data['fare_amount'] > 0) &
    (data['trip_distance'] > 0)
]

In [ ]:
#Convert pickup and dropoff times to datetime for easier analysis
cleaned_data['tpep_pickup_datetime'] = pd.to_datetime(cleaned_data['tpep_pickup_datetime'])
cleaned_data['tpep_dropoff_datetime'] = pd.to_datetime(cleaned_data['tpep_dropoff_datetime'])

#get hour, day of week and trip duration
cleaned_data['pickup_hour'] = cleaned_data['tpep_pickup_datetime'].dt.hour
cleaned_data['pickup_day'] = cleaned_data['tpep_pickup_datetime'].dt.day_name()
cleaned_data['trip_duration'] = (cleaned_data['tpep_dropoff_datetime'] - cleaned_data['tpep_pickup_datetime']).dt.total_seconds() / 60

In [ ]:
#Store the cleaned data in a database
conn = sqlite3.connect('yellow_cab.db')
cleaned_data.to_sql('cleaned_trips', conn, if_exists = 'replace', index = False)

In [ ]:
#Analyze Peak Hours
plt.figure(figsize = (10, 5))
sns.countplot(x = 'pickup_hour', data = cleaned_data, palette = 'viridis')
plt.title('Number of Trips by Hour')
plt.xlabel('Hour of the Day')
plt.ylabel('Number of Trips')
plt.show()

In [ ]:
#Analyze Popular Pickup Days
plt.figure(figsize = (10, 5))
sns.countplot(x = 'pickup_day', data = cleaned_data, order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.title('Number of Trips by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Number of Trips')
plt.show()

In [ ]:
#Analyze Fare Amount Distribution
plt.figure(figsize = (10, 5))
sns.histplot(cleaned_data['fare_amount'], bins = 50, kde = True)
plt.title('Fare Amount Distribution')
plt.xlabel('Fare Amount ($)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
#Correlation Heatmap for Numerical Features
plt.figure(figsize = (10, 5))
sns.heatmap(cleaned_data[['fare_amount', 'trip_distance', 'trip_duration']].corr(), annot = True, cmap = 'coolwarm', vmin = -1, vmax = 1)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
#Generate a final Report
summary = cleaned_data.describe()
print("Summary Statistics of Cleaned Data:")
print(summary)

#Close the database connection
conn.close()